# Example notebook translating OpenPTV files for Paraview using flowtracks

In [8]:
import numpy as np
import scipy
import pandas as pd
from flowtracks.io import trajectories_ptvis

In [9]:
dataset = trajectories_ptvis('./openptv_results/ptv_is.%d',xuap=False)

In [10]:
dataframes = []
for traj in dataset:
    dataframes.append(
        pd.DataFrame.from_records(
            traj,
            columns=['x','y','z','Vx','Vy','Vz','frame','Particle']
        )
    )

In [11]:
df = pd.concat(dataframes,ignore_index=True)

In [12]:
df['Particle'] = df['Particle'].astype(np.int32)

# Paraview does not recognize it as a set without _000001.txt, so we the first 10000
# ptv_is.10001 is becoming ptv_00001.txt

df['frame'] = df['frame'].astype(np.int32) - 10000

In [13]:
df.reset_index(inplace=True, drop=True)
df.head()

,x,y,z,Vx,Vy,Vz,frame,Particle
0,-0.004908,0.000015,-0.069549,-0.000037,0.000186,-0.000242,1,26
1,-0.004945,0.000201,-0.069791,-0.000032,0.000157,-0.000173,2,26
2,-0.004977,0.000358,-0.069964,0.000000,0.000000,0.000000,3,26
3,-0.005275,0.010814,-0.073490,-0.000026,-0.000033,-0.000105,1,252
4,-0.005301,0.010781,-0.073595,-0.000049,0.000010,-0.000150,2,252


In [14]:
df_grouped = df.reset_index().groupby('frame')
for index, group in df_grouped:
    group.to_csv(
        f'./paraview_output/ptv_{int(index):05d}.txt',
        mode='w',
        columns=['Particle','x','y','z','Vx','Vy','Vz'], 
        index=False
        )